**1. Загрузим набор данных с помощью библиотеки Corus**

Установим библиотеку Corus

In [1]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Сначала скачаем dataset.

In [2]:
!wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

--2023-04-27 08:58:54--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... 95.129.138.2
Connecting to ai-center.botik.ru (ai-center.botik.ru)|95.129.138.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3363777 (3.2M) [application/zip]
Saving to: ‘Persons-1000.zip.1’

Persons-1000.zip.1  100%[===================>]   3.21M  2.19MB/s    in 1.5s    

2023-04-27 08:58:56 (2.19 MB/s) - ‘Persons-1000.zip.1’ saved [3363777/3363777]



В самом низу предыдущей строки прописано имя сохранённого файла. Сейчас это и будет наш путь.

Используем corus для загрузки.

In [3]:
from corus import load_persons # Для загрузки 1000 персон. Прописано в библиотеке.

In [4]:
path = 'Persons-1000.zip'        # Указатель пути
records = load_persons(path)   # Загрузим одну часть
next(records)                       # Вернём эту часть

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [5]:
dataset=[next(records).text for i in range(999)]

dataset[0]

'Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, о

Возьмём все записи и запишем их в dataset. 
P.S. Если вы меняетеколичество записываемых значений, значит нужно заново выполнять код выше на строчку, где прописывается путь. И количество не должно превышать количество значенй в загруженном датасете.

In [6]:
dataset[:2]     # Посмотрим первые 3 значения

['Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.\r\n\r\n"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.\r\n\r\nПо сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".\r\n\r\nКомиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, 

Создадим копии dataset, что бы в каждом случае использовать свой.

In [7]:
dataset_nltk = dataset.copy()
dataset_pymorphy2 = dataset.copy()
dataset_mystem = dataset.copy()
dataset_spacy = dataset.copy()
dataset_natasha = dataset.copy()

**2. Проведём нормализацию.**

1.Уберём HTML разметку, если она у нас есть. С помошью библиотеки BeautifulSoup.

Интересное описание тут https://docs-python.ru/packages/paket-beautifulsoup4-python/

In [8]:
from bs4 import BeautifulSoup     # Импортируем BeautifulSoup

Для визуализации добавим неправильный элемент HTML в начало dataset, затем обработаем парсером, посмотрим на результат и уберём неправильный код. В итоге получим обработанный парсером наш dataset.

In [9]:
new_dataset=[]                              # Cоздадим новый список
#new_dataset= dataset
dataset.insert(0,"<a></p>")                 # Добавим неправильный код html в начало нашего списка


print('\n\n ***неправильный код html в начале наше списка dataset[0] = ',dataset[0])   # Проверим неправильный код html в начале нашего списка

for data in dataset :                       # Пройдёмся по всему dataset
  soup= BeautifulSoup(data,'html.parser')   # применим к data парсер html.parser 
  new_dataset.append(str(soup))             # сохраним в new_dataset новое значение data
  
print('\n\n ***Проверим исправленный код new_dataset[0] = ',new_dataset[0])   # Проверим неправильный код html в начале нашего списка

del new_dataset[0]                          # Удалим после этого этот элемент
print('\n\n ***После удаления кода HTML new_dataset[0] = ',new_dataset[0])    # Посмотрим значение после удаления.

dataset = new_dataset   # Сохраняем обработанный результат обратно.
print('\n\n ***После обработки dataset[0] = ',dataset[0])    # Посмотрим значение после удаления.



 ***неправильный код html в начале наше списка dataset[0] =  <a></p>


 ***Проверим исправленный код new_dataset[0] =  <a></a>


 ***После удаления кода HTML new_dataset[0] =  Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах

05/08/2008 10:32

МОСКВА, 5 августа /Новости-Грузия/.  Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.

"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.

По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".

Комиссар не называет конкретных стран, одно

А теперь объеденим наш dataset  в один список, добавим в начало неправильный элемент, обработаем парсером, получим текст и посмотрим результат. В итоге получим обработанный начальный список.

In [10]:
list_dataset = ' '.join(dataset)    # Обьеденим в общий список наш  dataset

list_dataset = "<a></p>" + list_dataset   # Добавим в наш список код HTML
print('\n\n list_dataset[:40] = ', list_dataset[:40])    # Посмотрим результат.

soup_list= BeautifulSoup(list_dataset,'html.parser')   # применим к list_dataset парсер html.parser 
list_dataset = str(soup_list.get_text())    # Занесём в наш списисок полусенный текст преобразованный в тип str.


print('\n\n list_dataset[:40] = ', list_dataset[:40])   # Посмотрим результат.



 list_dataset[:40] =  <a></p>Комиссар СЕ критикует ограничител


 list_dataset[:40] =  Комиссар СЕ критикует ограничительную по


2.Уберём CSS разметку, если она у нас есть. С помошью библиотеки BeautifulSoup для нашего списка.

In [11]:

for data in soup_list(['style', 'script']):
  data.decompose()
list_dataset = ' '.join(soup_list.stripped_strings)

3.Уберём Emoji если онb у нас есть. С помошью библиотеки Clean для нашего списка.

In [12]:
!pip install clean-text    # Установим библиотеку clean

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from unidecode import unidecode

In [15]:
from cleantext import clean   # Импортируем метод clean

emoji_dataset = list_dataset    # сохраним наш текст

emoji_dataset=clean(emoji_dataset, no_emoji=True )

**3. Проведём токенизацию**

3.1. Используем **NLTK**

In [16]:
import nltk   # Импортируем nltk
nltk.download('punkt')    # Загрузим punkt

from nltk.tokenize import (
    sent_tokenize,    # Токенизация по предложения
    word_tokenize,    # Токенизация по словам
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


3.1.1 Разбиение на предложения используя
**Sent_tokenize**

In [17]:
# Применим токенизацию на предложения, с указанием параметра что язык русский
nltk_sent_rus_list_dataset = nltk.sent_tokenize(list_dataset, language = 'russian')   

print('Всего получено предложений : ', len(nltk_sent_rus_list_dataset),'\n')
print('Разделение на предложения, с указанием языка : \n')
nltk_sent_rus_list_dataset[:5]

Всего получено предложений :  12532 

Разделение на предложения, с указанием языка : 



['Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских странах\r\n\r\n05/08/2008 10:32\r\n\r\nМОСКВА, 5 августа /Новости-Грузия/.',
 'Проводимая в европейских странах ограничительная политика в отношении беженцев нарушает ряд международных стандартов, в частности, право на воссоединение семей, заявляет Комиссар Совета Европы по правам человека Томас Хаммарберг (Thomas Hammarberg) в размещенном на его сайте еженедельном комментарии.',
 '"Ограничительная политика в отношении беженцев в европейских странах уменьшает возможности воссоединения разделенных семей", - полагает он.',
 'По сообщению РИА Новости, Хаммарберг констатирует, что в последнее время "правительства попытались ограничить приезд близких родственников к тем беженцам, которые уже проживают в стране".',
 'Комиссар не называет конкретных стран, одновременно отмечая, что в ряде случаев подобная линия привела "к неоправданным человеческим страданиям, когда члены семьи, зависящие друг от друга, оказал

3.2. Разбиение на предложения используя
**Word_tokenize**

In [18]:
# Применим токенизацию на слова, с указанием параметра что язык русский
nltk_word_rus_list_dataset = nltk.word_tokenize(list_dataset, language = 'russian')   

print('Всего получено слов : ', len(nltk_word_rus_list_dataset),'\n')
print('Разделение на слова, с указанием языка : \n')
nltk_word_rus_list_dataset[:5]

Всего получено слов :  259594 

Разделение на слова, с указанием языка : 



['Комиссар', 'СЕ', 'критикует', 'ограничительную', 'политику']

3.3. Разделим на слова используя пробелы в качестве разделителя с помщью функции **WhitespaceTokenizer**

In [19]:
from nltk.tokenize import WhitespaceTokenizer

nltk_space_rus_list_dataset = WhitespaceTokenizer().tokenize(list_dataset)   

print('Всего получено слов : ', len(nltk_space_rus_list_dataset),'\n')
print('Разделение на слова по пробелам: \n')
nltk_space_rus_list_dataset[:5]


Всего получено слов :  220682 

Разделение на слова по пробелам: 



['Комиссар', 'СЕ', 'критикует', 'ограничительную', 'политику']

Разделим на слова используя пунктуацию в качестве разделителя с помщью функции **WordPunctTokenizer**

In [20]:
from nltk.tokenize import WordPunctTokenizer

nltk_punct_rus_list_dataset = WordPunctTokenizer().tokenize(list_dataset)   

print('Всего получено слов : ', len(nltk_punct_rus_list_dataset),'\n')
print('Разделение на слова по пунктуации: \n')
nltk_punct_rus_list_dataset[:5]

Всего получено слов :  268993 

Разделение на слова по пунктуации: 



['Комиссар', 'СЕ', 'критикует', 'ограничительную', 'политику']

3.4. Для примера, разделим слова на слоги, и посмотрим что получилось.
Используем **SyllableTokenizer**.

In [21]:
from nltk.tokenize import SyllableTokenizer

syll_tokenizer = SyllableTokenizer(
    lang='ru',    # Язык русский
    sonority_hierarchy=[
        'аеёиоуыэюя',   # гласные
        'мн',   # носовые
        'вфзсжшх',    # фрикативные
        'рпбдткгщ',   # стоповые
    ]
)

nltk_syll_rus_list_dataset=[]

for data in nltk_punct_rus_list_dataset:    # Прогоним по нашему датасету.
  nltk_syll_rus_list_dataset.append(syll_tokenizer.tokenize(data))    # Добавить в новый датасет раздённые слова по слогам.


print('Всего получено слов : ', len(nltk_syll_rus_list_dataset),'\n')
print('Разделение на слоги: \n')
nltk_syll_rus_list_dataset[:15]

/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'ч'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'л'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'ь'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'ц'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Character not defined in sonority_hierarchy, assigning as vowel: 'й'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/nltk/tokenize/sonority_sequencing.py:102: UserWarning: Charac

Всего получено слов :  268993 

Разделение на слоги: 



[['Ко', 'мис', 'сар'],
 ['СЕ'],
 ['кри', 'ти', 'кует'],
 ['ог', 'ра', 'нич', 'и', 'тел', 'ь', 'ную'],
 ['пол', 'и', 'ти', 'ку'],
 ['в'],
 ['о', 'тно', 'ше', 'нии'],
 ['бе', 'же', 'нцев'],
 ['в'],
 ['ев', 'ро', 'пейс', 'ких'],
 ['стра', 'нах'],
 ['05'],
 ['/'],
 ['08'],
 ['/']]

3.2 Используем **Spacy**

In [22]:
import spacy    # Импортируем spacy

In [23]:
!python3 -m spacy download ru_core_news_sm    # Скачаем Russian pipeline optimized

2023-04-27 08:59:47.257689: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 43.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [24]:
nlp = spacy.load("ru_core_news_sm")
nlp.max_length =10**8   #увеличим максимальную длинну 

nlp_list_dataset = nlp(list_dataset)    # Получим результат
nlp_list_dataset[:10]


Комиссар СЕ критикует ограничительную политику в отношении беженцев в европейских

**4. Удаление стоп слов. Используем NLTK.**

In [25]:
#import nltk    # Не забываем про это. У нас уже раньше использовалось.
nltk.download('stopwords')    # Cкачаем стоп слова

from nltk.corpus import stopwords
rus_stop_words = stopwords.words("russian")    # Получим русские стоп слова.



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
nostopword_nltk_punct_rus_list_dataset=[]

print('Количество токенов в датасете до удаления до удаления стоп слов : ', len(nltk_punct_rus_list_dataset))

for token1 in nltk_punct_rus_list_dataset:    # Пройдемся по датасету
    if token1 not in rus_stop_words:    # Проверим на стоп слова
        nostopword_nltk_punct_rus_list_dataset.append(token1)

print('Количество токенов в датасете после удаления стоп слов : ', len(nostopword_nltk_punct_rus_list_dataset))

Количество токенов в датасете до удаления до удаления стоп слов :  268993
Количество токенов в датасете после удаления стоп слов :  224356


**5. Лемматизация**

5.1. Используем **pymorphy2**

In [27]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import pymorphy2    # Импортируем
morph = pymorphy2.MorphAnalyzer()   # Создадим модель
pym2_nostopword_nltk_punct_rus_list_dataset =[]   

for doc in nostopword_nltk_punct_rus_list_dataset:   # Пройдём по всему датасету
  pym2_nostopword_nltk_punct_rus_list_dataset.append(morph.parse(doc)[0].normal_form)   # Положим в новый датасет полученные результаты.

pym2_nostopword_nltk_punct_rus_list_dataset[:25]    # Посмотрим 10 значений

['комиссар',
 'сие',
 'критиковать',
 'ограничительный',
 'политика',
 'отношение',
 'беженец',
 'европейский',
 'страна',
 '05',
 '/',
 '08',
 '/',
 '2008',
 '10',
 ':',
 '32',
 'москва',
 ',',
 '5',
 'август',
 '/',
 'новость',
 '-',
 'грузия']

4.2 Используем **Spacy**

In [29]:
spasy_lemma_nostopword_nlp_list_dataset=[]

for token in nlp_list_dataset:    # Проходим по датасету, которй раньше уже был сделан nlp.
  if token.is_stop == False :   # Если это не стопслово
    spasy_lemma_nostopword_nlp_list_dataset.append(token.lemma_)    # Тогда запишем в новый датасет лемматизированную форму данного слова.
    
spasy_lemma_nostopword_nlp_list_dataset[:15]    # Посмотрим 15 значений))


['комиссар',
 'критиковать',
 'ограничительный',
 'политика',
 'отношение',
 'беженец',
 'европейский',
 'страна',
 '\r\n\r\n',
 '05/08/2008',
 '10:32',
 '\r\n\r\n',
 'москва',
 ',',
 '5']

In [30]:
print ('Итоговый dataset с использованием pymorphy2 :\n')
print (pym2_nostopword_nltk_punct_rus_list_dataset[:10],'\n')
print (pym2_nostopword_nltk_punct_rus_list_dataset[10:20],'\n\n')

print ('Итоговый dataset с использованием spacy :\n')
print (spasy_lemma_nostopword_nlp_list_dataset[:10],'\n')
print (spasy_lemma_nostopword_nlp_list_dataset[10:20],'\n\n')

Итоговый dataset с использованием pymorphy2 :

['комиссар', 'сие', 'критиковать', 'ограничительный', 'политика', 'отношение', 'беженец', 'европейский', 'страна', '05'] 

['/', '08', '/', '2008', '10', ':', '32', 'москва', ',', '5'] 


Итоговый dataset с использованием spacy :

['комиссар', 'критиковать', 'ограничительный', 'политика', 'отношение', 'беженец', 'европейский', 'страна', '\r\n\r\n', '05/08/2008'] 

['10:32', '\r\n\r\n', 'москва', ',', '5', 'август', '/новости', '-', 'грузия/.', ' '] 


